In [1]:
from copy import deepcopy
import numpy as np
import pandas as pd
import calendar
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus import AbstractRetrieval

from tqdm import tqdm

pd.options.display.max_columns = 30

filename = "./energy_factory.txt"

In [2]:
df_re = pd.read_pickle("./df_factory.pkl")
df_re.head(3)

,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,affilname,affiliation_city,affiliation_country,author_count,author_names,author_ids,...,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,fund_acr,fund_no,fund_sponsor
0,2-s2.0-85111496548,10.1186/s12934-021-01639-7,None,None,Enforcing ATP hydrolysis enhanced anaerobic gl...,ar,Article,Dai Z.,60027363;60021016;60019499;60012932,University of Chinese Academy of Sciences;Inst...,Beijing;Beijing;Beijing;Clayton,China;China;China;Australia,6,"Dai, Zongjie;Zhu, Yan;Dong, Hongjun;Zhao, Chun...",36089670400;57189383541;35603311500;5695965610...,...,None,18593,14752859,Journal,20,1,149,None,Background: The intracellular ATP level is an ...,ABE fermentation | Acidogenesis | Anaerobic fe...,0,1,NSFC,31670048,National Natural Science Foundation of China
1,2-s2.0-85110191717,10.1016/j.jobe.2021.102955,S2352710221008135,None,Seismic fragility analysis of steel intermedia...,ar,Article,Yu E.,60024872;60012291;60000872,Hanyang University;Incheon National University...,Seoul;Incheon;Chuncheon,South Korea;South Korea;South Korea,3,"Yu, Eunjong;Kim, Taewan;Park, Ji Hun",8639910700;57194881369;32867930500,...,None,21100389518,23527102,Journal,44,None,102955,None,Steel intermediate moment frames (SIMFs) with ...,Beam splice slip | Collapse probability | Colu...,0,0,None,undefined,None
2,2-s2.0-85101125317,10.1038/s41598-020-80217-0,None,33574347,Bond order redefinition needed to reduce inher...,ar,Article,Syuhada I.,60069382,Institut Teknologi Bandung,Bandung,Indonesia,6,"Syuhada, Ibnu;Hauwali, Nikodemus Umbu Janga;Ro...",57189732352;57211662136;57189732507;3616070250...,...,None,21100200805,20452322,Journal,11,1,3674,None,"In this work, we present the bond order redefi...",None,0,1,None,undefined,None


In [3]:
df_ab = pd.read_pickle("./df_ab.pkl")
df_ab.shape

(1017, 28)

In [4]:
# publication type
# "J" = Journal
# "B" = Book
# "S" = Series
# "P" = Patent
dict_pubtype = {'ar': 'J',    # 'Article',
                'ab': 'J',    # 'Abstract Report',
                'bk': 'B',    # 'Book',
                'ch': 'B',    # 'Book Chapter',
                'bz': 'J',    # 'Business Article',
                'cp': 'J',    # 'Conference Paper',
                'cr': 'J',    # 'Conference Review',
                'dp': 'J',    # 'Data Paper ',
                'ed': 'S',    # 'Editorial',
                'er': 'J',    # 'Erratum',
                'le': 'J',    # 'Letter',
                'no': 'S',    # 'Note',
                'pr': 'S',    # 'Press Release',
                'rp': 'J',    # 'Report',
                'tb': 'J',    # 'Retracted',
                're': 'J',    # 'Review',
                'sh': 'J',    # 'Short Survey'
                'ip': 'P'     # 'Patent' 
               }

# documentation type
dict_docutype = {'ar': 'Article',
                 'ab': 'Abstract Report',
                 'bk': 'Book',
                 'ch': 'Book Chapter',
                 'bz': 'Business Article',
                 'cp': 'Conference Paper',
                 'cr': 'Conference Review',
                 'dp': 'Data Paper ',
                 'ed': 'Editorial',
                 'er': 'Erratum',
                 'le': 'Letter',
                 'no': 'Note',
                 'pr': 'Press Release',
                 'rp': 'Report',
                 'tb': 'Retracted',
                 're': 'Review',
                 'sh': 'Short Survey',
                 'ip': 'Patent'
                }

# text cleaning
dict_clean_text = {"&amp;": "&", 
                   "&nbsp;": " ", 
                   "&lt;": "<",
                   "&gt;": ">"
                  }
def get_clean_text(text):
    for k, v in dict_clean_text.items():
        text = text.replace(k, v)
    return text

In [5]:
# all

import os, calendar

with open(filename, "w") as datafile:
    datafile.write("FN Clarivate Analytics Web of Science\nVR 1.0")
    
    for art in df_ab.index:
        df_row = df_ab.loc[art]
        
        # 1. PT: publication type
        if df_row['PT']:
            pubtype_ = dict_pubtype[df_row['PT']]
        else:
            pubtype_ = "J"
        datafile.write(f"\nPT {pubtype_}")
        
        # 2. AU: author names
        author_name_ = df_row['AU']
        author_name = "\nAU "
        if all(author_name_):
            author_name += "\n   ".join(author_name_)
            
        datafile.write(author_name)
        
        # 3. AF: affiliations
        affiliation_ = df_row['AF']
        affiliation = "\nAF "
        if all(affiliation_):
            affiliation += "\n   ".join(affiliation_)
        datafile.write(affiliation)
        
        # 4. TI: document title
        title = '\nTI '
        title_ = df_row['TI']
        if title_:
            title += get_clean_text(title_)
        datafile.write(title)
        
        # 5. SO: publication name
        so = '\nSO '
        so_ = df_row['SO']
        if so_:
            so += get_clean_text(so_)
        datafile.write(so)
        
        # 6. LA : Language
        datafile.write("\nLA " + df_row["LA"])
        
        # 7. DT : Document Type
        if df_row['DT']:
            docutype_ = dict_docutype[df_row['DT']]
        else:
            docutype_ = "J"
        datafile.write(f"\nDT {docutype_}")
        
        # 8. DE : Author Keywords
        de_ = df_row['DE']
        datafile.write(f"\nDE {de_}")
        
        # 9. ID : Keyword Plus
        id_ = df_row['ID']
        datafile.write(f"\nID {id_}")
        
        # 10. AB: Abstract
        ab_ = df_row['AB']
        datafile.write(f"\nAB {ab_}")
        
        # 11. C1 : Author Address
        c1_ = df_row['C1']
        c1 = [f"[{'; '.join(c[1])}] {', '.join(c[2].split(', ')[1:])}" for c in c1_]
        c1 = ".\n   ".join(c1) + "."
        datafile.write("\nC1 " + c1)
        
        # 12. RP : Reprint Address
        datafile.write("\nRP None")

        # 13. EM : E-mail Address
        datafile.write("\nEM None")
        
        # 14. CR : Cited References
        cr_ = df_row["CR"]
        if len(cr_) > 0:
            cr_[0] = "  " + cr_[0][2:]
            cr_ = [c for c in cr_ if len(c) > 5]
            cr_ = [c.lstrip(" ") for c in cr_ if (c[3] != ',') or ("DOI" in c)]
        datafile.write("\nCR "+"\n   ".join(cr_))
        
        # 15. NR : Cited Reference Count
        datafile.write(f"\nNR {df_row['NR']}")
        
        # 16. TC : Web of Science Core Collection Times Cited Count
        datafile.write(f"\nTC {df_row['TC']}")
        
        # 17. Z9 : Total Times Cited Count
        z9_ = df_row['Z9']
        datafile.write(f"\nZ9 {z9_}")
        
        # 18. U1 : Usage Count (Last 180 Days)
        # 19. U2 : Usage Count (Since 2013)
        # 20. PU : Publisher = ELSEVIER SCI LTD
        # 21. PI : Publisher City = OXFORD
        # 22. PA : Publisher Address = THE BOULEVARD, LANGFORD LANE, KIDLINGTON, OXFORD OX5 1GB, OXON, ENGLAND
        # 23. SN : International Standard Serial Number (ISSN) = 0959-6526
        sn_ = df_row["SN"]
        datafile.write(f"\nSN {sn_}")
        
        # 24. EI : Electronic International Standard Serial Number (eISSN) = 1879-1786
        # 25. J9 : 29-Character Source Abbreviation = J CLEAN PROD
        j9_ = df_row["J9"]
        datafile.write(f"\nJ9 {j9_}")
        
        # 26. JI : ISO Source Abbreviation = J. Clean Prod.
        ji_ = df_row["JI"]
        datafile.write(f"\nJI {ji_}")
        
        # 27. PD : Publication Date = JUL 1
        pd_ = df_row["PD"]
        datafile.write(f"\nPD {pd_}")
        
        # 28. PY : Publication Year = 2020
        py_ = df_row["PY"]
        datafile.write(f"\nPY {py_}")
        
        # 29. VL : Volumn = 260
        vl_ = df_row["VL"]
        datafile.write(f"\nVL {vl_}")
        
        # 30. AR : Article Number = 121059
        ar_ = df_row["AR"]
        datafile.write(f"\nAR {ar_}")
        
        # 31. DI : Digital Object Identifier = 10.1016/j.jclepro.2020.121059
        doi_ = df_row["DI"]
        datafile.write(f"\nDI {doi_}")
        
        # 32. PG : Page Count = 14
        # 33. WC : Web of Science Categories = Green & Sustainable Science & Technology; Engineering, Environmental; Environmental Sciences
        # 34. SC : Research Areas = Science & Technology - Other Topics; Engineering; Environmental Sciences & Ecology
        sc_ = df_row["SC"]
        datafile.write(f"\nSC {sc_}")
        
        # end 
        datafile.write(f"\nER\n")
        
    datafile.write("\nEF\n")